Предположим, что асессор за 30 секунд своего рабочего времени получает N рублей.

Какую оплату вы считаете справедливой для выполнения асессором одного микрозадания из этого файла? Опишите подробно все этапы вашего решения.

In [1]:
#импорт библиотек
import numpy as np 
import pandas as pd
from datetime import datetime

In [2]:
#считываем файл и сохраняем в переменную data
path = 'E:\task1.txt'# путь к папке на локальном компьютере
data = pd.read_csv(path, sep='\t')
data.head(10)

,login,tid,Microtasks,assigned_ts,closed_ts
0,login0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01
1,login0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29
2,login0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10
3,login0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56
4,login0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50
5,login0,190560934.0,1.0,2017-04-20 12:28:53,2017-04-20 13:15:23
6,login0,190563590.0,1.0,2017-04-20 12:29:06,2017-04-20 13:15:36
7,login0,190565094.0,1.0,2017-04-20 13:47:43,2017-04-20 14:19:46
8,login0,190560460.0,1.0,2017-04-20 14:02:13,2017-04-20 14:22:22
9,login0,190662496.0,5.0,2017-04-20 15:22:24,2017-04-20 17:22:10


In [3]:
#Общая информация о данных таблицы data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701827 entries, 0 to 701826
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   login        701827 non-null  object 
 1   tid          701827 non-null  float64
 2   Microtasks   701827 non-null  float64
 3   assigned_ts  701827 non-null  object 
 4   closed_ts    701827 non-null  object 
dtypes: float64(2), object(3)
memory usage: 26.8+ MB


Рассмотрим полученную информацию подробнее.

Пропущенных значений нет.

Всего в таблице 5 столбцов, тип данных у большинства - object.


Подробно разберём, какие в data столбцы и какую информацию они содержат:

login — логин асессора; 

tid — id оцениваемого задания (task id); 

Microtasks – количество микрозаданий в одном задании; 

assigned_ts — время резервирования системой задания для асессора; 

closed_ts — точное время завершения работы над заданием

поменяем тип Microtasks и tid на 'int', т.к. это целочисленные значения, а при выгрузке датасета они float

In [4]:
data['tid'] = data['tid'].astype('int')
data['Microtasks'] = data['Microtasks'].astype('int')

переведем столбцы с датой в нужный формат и добавим столбец с суммарным количеством потраченного на задание времени (в секундах)

In [5]:
data['assigned_ts'] = pd.to_datetime(data['assigned_ts'])
data['closed_ts'] = pd.to_datetime(data['closed_ts'])
data['time_for_tasks'] = (data['closed_ts'] - data['assigned_ts']).dt.total_seconds()
data['time_for_one_task'] = data['time_for_tasks']/data['Microtasks']
data.head(10)

,login,tid,Microtasks,assigned_ts,closed_ts,time_for_tasks,time_for_one_task
0,login0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01,3802.0,950.50
1,login0,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29,1079.0,1079.00
2,login0,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10,4119.0,1029.75
3,login0,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56,2726.0,2726.00
4,login0,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50,2768.0,1384.00
5,login0,190560934,1,2017-04-20 12:28:53,2017-04-20 13:15:23,2790.0,2790.00
6,login0,190563590,1,2017-04-20 12:29:06,2017-04-20 13:15:36,2790.0,2790.00
7,login0,190565094,1,2017-04-20 13:47:43,2017-04-20 14:19:46,1923.0,1923.00
8,login0,190560460,1,2017-04-20 14:02:13,2017-04-20 14:22:22,1209.0,1209.00
9,login0,190662496,5,2017-04-20 15:22:24,2017-04-20 17:22:10,7186.0,1437.20


In [6]:
#Посмотрим, есть ли одинаковые задания у разных асессоров
len(data['tid'].unique())

635044

In [7]:
#Найдем количество асессоров, представленных в данном файле
len(data['login'].unique())

767

In [8]:
tid_counts = data.tid.value_counts()
tid_counts.head(10)

171470568    134
188676794    118
195476986     96
196322338     86
171266128     85
184144118     81
184144104     80
184144114     80
184144116     80
184144100     80
Name: tid, dtype: int64

Видим, что задание 171470568 выполнило наибольшее число пользователей. 

Чтобы избежать переплаты за время, которое пользователь мог не тратить на выполнение задания после его назначения, возьмем по каждому заданию минимальное время выполнения среди всех асессоров.

Посмотрим, сколько заданий выполнялись более чем 1 асессором:

In [9]:
sum(data.tid.value_counts() > 1)

29627

In [10]:
data_pivot = data.pivot_table(columns = 'tid', values = 'time_for_tasks', aggfunc=np.min)
data_pivot = data_pivot.transpose()
data_pivot.head(10)

,time_for_tasks
tid,
113958641,836.0
113958643,107.0
113958647,60.0
113958649,156.0
113958653,699.0
113958655,362.0
113958657,574.0
113958659,58.0
113958661,143.0


In [11]:
#Объединим две таблицы по значению tid
res = data.merge(data_pivot, on=["tid"])
res.head(10)

,login,tid,Microtasks,assigned_ts,closed_ts,time_for_tasks_x,time_for_one_task,time_for_tasks_y
0,login0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01,3802.0,950.50,3802.0
1,login0,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29,1079.0,1079.00,1079.0
2,login0,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10,4119.0,1029.75,4119.0
3,login0,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56,2726.0,2726.00,2726.0
4,login0,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50,2768.0,1384.00,2768.0
5,login0,190560934,1,2017-04-20 12:28:53,2017-04-20 13:15:23,2790.0,2790.00,2790.0
6,login0,190563590,1,2017-04-20 12:29:06,2017-04-20 13:15:36,2790.0,2790.00,2790.0
7,login0,190565094,1,2017-04-20 13:47:43,2017-04-20 14:19:46,1923.0,1923.00,1923.0
8,login0,190560460,1,2017-04-20 14:02:13,2017-04-20 14:22:22,1209.0,1209.00,1209.0
9,login0,190662496,5,2017-04-20 15:22:24,2017-04-20 17:22:10,7186.0,1437.20,7186.0


In [12]:
(res['time_for_tasks_x'] == res['time_for_tasks_y']).value_counts()

True     655014
False     46813
dtype: int64

In [13]:
res['time_for_one_task_y'] = res['time_for_tasks_y']/res['Microtasks']
res

,login,tid,Microtasks,assigned_ts,closed_ts,time_for_tasks_x,time_for_one_task,time_for_tasks_y,time_for_one_task_y
0,login0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01,3802.0,950.500000,3802.0,950.500000
1,login0,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29,1079.0,1079.000000,1079.0,1079.000000
2,login0,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10,4119.0,1029.750000,4119.0,1029.750000
3,login0,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56,2726.0,2726.000000,2726.0,2726.000000
4,login0,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50,2768.0,1384.000000,2768.0,1384.000000
...,...,...,...,...,...,...,...,...,...
701822,login766,195580810,4,2017-05-15 12:24:26,2017-05-15 14:09:36,6310.0,1577.500000,6310.0,1577.500000
701823,login766,195580162,1,2017-05-15 12:24:33,2017-05-15 14:10:18,6345.0,6345.000000,6345.0,6345.000000
701824,login766,195584796,1,2017-05-15 12:24:41,2017-05-15 14:11:10,6389.0,6389.000000,6389.0,6389.000000
701825,login766,195656336,3,2017-05-15 15:54:18,2017-05-15 16:32:11,2273.0,757.666667,2273.0,757.666667


In [14]:
#проверим данные на количество дубликатов. Если найдутся, удаляем, и проверяем, все ли удалились.
res.duplicated().sum()

21790

In [15]:
res = res.drop_duplicates().reset_index(drop=True)
res.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Их удаление позволит провести анализ точнее.

Вычислим на какое значение нужно умножать стоимость N работы за 30 секунд, чтобы получить итоговую сумму за одно микрозадание 

Посчитаем затраченное на выполнение одного задания время, просуммировав время одного задания всех пользователей и раделив на временной тайминг оплаты - 30 секунд

In [16]:
time_coeff = (((res.groupby('login')['time_for_tasks_y'].sum()))/30)/res['Microtasks'].sum()


Теперь вычислим итоговую сумму за одно микрозадание:

В среднем ассессоры в Яндексе получают з/п примерно 25000рублей/месяц при работе 40 часов в неделю при графике 5/2. Вычислим, какая сумма получится за 30 секунд рабочего времени:

25000рублей / (20раб.дней * 8раб.часов * 60минут * 60сек / 30сек) = 1,30208333рубля

В итоге, с учетом среднего затраченного на задание пользователем времени и средней з/п ассессора, получаем следующую примерную стоимость одного микрозадания:

In [17]:
N = 1.3
payment = N * time_coeff
payment.sum()/len(payment)

0.02839337550837709

Таким образом, примерная стоимость одного микрозадания из данного файла составляет 0.02839рублей